## **Embeddings**

![skpatternlarge](./imgs/01/skpatternlarge.png)

在 OpenAI 中，Embeddings 技术的应用非常广泛，将文本字符串转换为浮点向量，通过向量之间的距离来衡量文本之间的相似度。不同行业希望加入自己的数据 我们就可以把这些企业级的数据通过 OpenAI Embeddings - text-embedding-ada-002 模型查询出向量，并通过映射进行保存，在使用时将问题也转换为向量，通过相似度的算法对比，找出最接近的 TopN 结果，从而找到与问题相关联的企业内容。

我们可以通过向量数据库将企业数据向量化后保存，结合 text-embedding-ada-002模型通过向量的相似度进行查询，从而找到与问题相关联的企业内容。现在常用的向量数据库就包括 Qdrant, Milvus, Faiss, Annoy, NMSLIB 等。

OpenAI 的文本嵌入向量文本字符串的相关性。 嵌入通常用于以下场景

- 搜索（结果按与查询字符串的相关性排序）
- 聚类（其中文本字符串按相似性分组）
- 推荐（推荐具有相关文本字符串的项目）
- 异常检测（识别出相关性很小的异常值）
- 多样性测量（分析相似性分布）
- 分类（其中文本字符串按其最相似的标签分类）

In [1]:
! pip install semantic-kernel -U
! pip install qdrant_client -U

In [2]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding
from semantic_kernel.connectors.memory.qdrant import QdrantMemoryStore

In [3]:
deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()

In [4]:
kernel = sk.Kernel()

In [5]:


kernel.add_chat_service("azure_chat_competion_service", AzureChatCompletion("GPT3TurboModel",endpoint,api_key=api_key,api_version = "2023-07-01-preview"))


kernel.add_text_embedding_generation_service(
        "embeddings_services", AzureTextEmbedding("EmbeddingModel", endpoint,api_key=api_key,api_version = "2023-07-01-preview")
)

In [6]:
qdrant_store = QdrantMemoryStore(vector_size=1536, url="http://localhost",port=6333)
await qdrant_store.create_collection_async('aboutMe')

In [7]:
kernel.register_memory_store(memory_store=qdrant_store)

In [8]:
base_vectordb = 'aboutMe'

In [9]:
await kernel.memory.save_information_async(base_vectordb, id="info1", text="卢建晖是微软高级云技术布道师")
await kernel.memory.save_information_async(base_vectordb, id="info2", text="卢建晖是前微软微软最有价值专家")
await kernel.memory.save_information_async(base_vectordb, id="info3", text="卢建晖是人工智能专家")
await kernel.memory.save_information_async(base_vectordb, id="info4", text="OpenAI 是非常热的人工智能公司")

In [10]:
ask = "卢建晖是谁？"

memories = await kernel.memory.search_async(
    base_vectordb, ask, limit=5, min_relevance_score=0.8
)

In [11]:
i = 0
for memory in memories:
    i = i + 1
    print(f"Top {i} Result: {memory.text} with score {memory.relevance}")

Top 1 Result: 卢建晖是人工智能专家 with score 0.8930044
Top 2 Result: 卢建晖是前微软微软最有价值专家 with score 0.8702849
Top 3 Result: 卢建晖是微软高级云技术布道师 with score 0.8636302
